In [ ]:
%load_ext aiida
%aiida
import pandas as pd
import urllib.parse as urlparse
import matplotlib.pyplot as plt

import ipywidgets as ipw
from aiida import orm
#from aiidalab_widgets_base import viewer

from surfaces_tools import helpers
from surfaces_tools.widgets import comments, obsolete

In [ ]:
pk = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)["pk"][0]
workcalc = orm.load_node(pk)
closest_50 =  workcalc.outputs.report.get_dict()['closest_to_50']
closest_60 =  workcalc.outputs.report.get_dict()['closest_to_60']
summary= workcalc.outputs.report.get_dict()['summary']
timings= workcalc.outputs.report.get_dict()['min_times_per_nnodes']

# Summary

In [ ]:
print(summary)

In [ ]:
label=ipw.Text(description='Label:', value='GEO_step')
multiplier=ipw.IntText(description='Multiplier:', value=20)
# button to crate report
button = ipw.Button(description="Report")
output = ipw.Output()
display(label, multiplier,button,output)

In [ ]:
nodes=[]
times=[]
speedup=[]
speedup_ideal=[]
speedup_50=[]
for nnodes, time_and_id in timings.items():
    nodes.append(int(nnodes))
    times.append(time_and_id[0])
speedup = [times[0]/i for i in times]
speedup_ideal = [i/nodes[0] for i in nodes]
efficiency = [i/(nodes[0]*times[0]) for i in times]
speedup_50 = [i/(2*nodes[0]) for i in nodes]
# on button click
def on_button_clicked(b):
    with output:
        output.clear_output()
        # Creating a dictionary with the data
        data = {
            "#nodes": [int(k) for k in timings.keys()],
            "time (s)": [time_and_id[0] for time_and_id in timings.values()],
            f'{label.value} (s)': [time_and_id[0]*multiplier.value for time_and_id in timings.values()],
            "speed-up": speedup,
            "efficiency": efficiency,
            "job_id": [time_and_id[1] for time_and_id in timings.values()],
        }
        # Creating a DataFrame
        df = pd.DataFrame(data)
        # Adding new rows for speedup targets directly to the DataFrame
        df.loc[len(df.index)] = [f'#nodes ~60% speedup: {closest_60}', "", "","","",""]
        df.loc[len(df.index)] = [f'#nodes ~50% speedup: {closest_50}', "", "","","",""]

        # Display the DataFrame
        display(df.style.hide(axis='index'))
        #display(df)

In [ ]:
# observe button click
button.on_click(on_button_clicked)

# Plot

In [ ]:
# Create plots with pre-defined labels.
fig, ax = plt.subplots()
ax.plot(nodes, speedup, 'ro-', label='benchmark speedup')
ax.plot(nodes, speedup_ideal, 'go-', label='ideal speedup')
ax.plot(nodes, speedup_50, 'yo-', label='50% ideal speedup')
ax.set_xlabel('# nodes')
ax.set_ylabel('speedup')
legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('C7')

plt.show()
#plt.savefig('speedup.png',dpi=300)

# Comments

In [ ]:
comments_widget = comments.CommentsWidget(workchain=pk)
display(comments_widget)

# Mark calculation as obsolete 

In [ ]:
obsolete = obsolete.ObsoleteWidget(workchain=pk)
display(obsolete)